# AIAp Miniproject 2

## Contents

- [Imports](#imports)

## Imports

In [29]:
import os
from pathlib import Path

from keras import utils as keras_utils
import tensorflow as tf

## Dataset

In [43]:
data_folder = Path(os.getcwd()).parent / "data"
raw_folder = data_folder / "raw" / "Architectural_Heritage_Elements_Dataset_64(creative_commons)_revised"

dataset = keras_utils.image_dataset_from_directory(raw_folder, image_size=(64, 64), batch_size=None)

print(dataset)




Found 10437 files belonging to 11 classes.
<ShuffleDataset element_spec=(TensorSpec(shape=(64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>


AttributeError: 'ShuffleDataset' object has no attribute 'shape'

### b) Citation and description

Citation: 
> Llamas, J. (2017). Architectural Heritage Elements Dataset: 128 (creative commons) (Version 1) [Dataset; .Jpg in .Zip]. Fundación CARTIF. https://correo.cartif.es/home/joslla@cartif.es/Briefcase/Architectural_Heritage_Elements_image_Dataset/Architectural_Heritage_Elements_Dataset_128%28creative_commons%29.zip

- **Title:** Architectural Heritage Elements Dataset
- **Subtitle:** 128 (creative commons)
- **Version:** 1
- **Publication Date:** 2017-02-20
- **Author:** Jose Llamas
- **Organization:** Fundación CARTIF
- **Source:** https://correo.cartif.es/home/joslla@cartif.es/Briefcase/Architectural_Heritage_Elements_image_Dataset/Architectural_Heritage_Elements_Dataset_128%28creative_commons%29.zip
- **Media:** .jpg in .zip
- **Download:** 2024-03-26

The raw data was extracted with Windows

### c) Splitting into training and test data

In [35]:
train_val_data, test_data = keras_utils.split_dataset(dataset, left_size=0.7, seed=64)

print(dataset)

6


## Exploratory data analysis

## Exploratory data analysis

## Architecture 1 (underfitting)

## Architecture 2 (overfitting)

## Architecture 2 (optimized)

## Quantification of the model performance

## Discussion